In [1]:
import pandas as pd
from rouge_score import rouge_scorer
import evaluate

/home/kilnaar/anaconda3/envs/ai574-pocs/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('./mtsamples.csv')
df

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."
...,...,...,...,...,...,...
4994,4994,Patient having severe sinusitis about two to ...,Allergy / Immunology,Chronic Sinusitis,"HISTORY:, I had the pleasure of meeting and e...",NaN
4995,4995,This is a 14-month-old baby boy Caucasian who...,Allergy / Immunology,Kawasaki Disease - Discharge Summary,"ADMITTING DIAGNOSIS: , Kawasaki disease.,DISCH...","allergy / immunology, mucous membranes, conjun..."
4996,4996,A female for a complete physical and follow u...,Allergy / Immunology,Followup on Asthma,"SUBJECTIVE: , This is a 42-year-old white fema...",NaN
4997,4997,Mother states he has been wheezing and coughing.,Allergy / Immunology,Asthma in a 5-year-old,"CHIEF COMPLAINT: , This 5-year-old male presen...",NaN


# ROUGE F1 Scores

In [54]:
rouge_calcer = rouge_scorer.RougeScorer([
    'rouge1', 
    'rouge2', 
    'rouge3',
    'rouge4',
    'rougeL',
    'rougeLsum'
], use_stemmer = True)

def calc_rouge(row):
    short_text, full_text = row['description'], row['transcription']        

    try:
        rouges = rouge_calcer.score(short_text, full_text)
        return rouges['rouge1'][2], rouges['rouge2'][2], rouges['rouge3'][2], rouges['rouge4'][2], rouges['rougeL'][2], rouges['rougeLsum'][2]
    except Exception as e:
        return 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [55]:
df[['rouge1_f1', 'rouge2_f1', 'rouge3_f1', 'rouge4_f1', 'rougeL_f1', 'rougeLsum_f1']] = df.apply(calc_rouge, axis = 1, result_type = 'expand')

# BERTScore

In [68]:
bertscore = evaluate.load("bertscore")

def calc_bertscore(row):
    short_text, full_text = row['description'], row['transcription'] 

    try:
        result = bertscore.compute(
                predictions = [short_text],
                references = [full_text],
                model_type = "microsoft/deberta-large-mnli",
                lang = "en")

        return result["f1"][0]
    except:
        return 0.0

In [69]:
df['bertscore_f1'] = df.head(90).apply(calc_bertscore, axis = 1)

# Save

In [71]:
df.to_csv('mtsamples_with_metrics.csv', index = False)